In [1]:
import pandas as pd
import numpy as np
#from feature_generators import TS_feature_generator_functions as fg
from timeit import default_timer as dt
from datetime import datetime
import re
from sklearn.externals import joblib
from time import time
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
data_dir ='/home/flowcast/data/python/data_validate_180315/'


In [5]:
ls /home/flowcast/data/python/data_validate_180315/

all_prod_180319.csv           data_3NF_180316_ECPR.csv
all_prod_validate_180319.csv  data_3NF_180316_IFNB.csv
buyer_supplier_3m.csv         data_3NF_180316_IFNS.csv
buyer_supplier_6m.csv         data_3NF_180316_LBDS.csv
buyer_supplier_all.csv        data_3NF_180316_LRRP.csv
buyer_supplier_prevyr.csv     data_3NF_180316_PSHP.csv
ccy_features_vector.csv       data_3NF_180316_RSRV.csv
ccy_features_vol.csv          data_3NF_180316_TRDS.csv
client_3m.csv                 data_CG.csv
client_6m.csv                 data_for_model.csv
client_all.csv                lr_dr_180223.joblib
client_prevyr.csv             lr_ld_180223.joblib
data_3NF_180316_BATB.csv      minmaxscaler_180223.pkl
data_3NF_180316_BDBR.csv      rf_dr_180223.joblib
data_3NF_180316_ECIF.csv      rf_ld_180223.joblib


In [6]:
data = pd.read_csv(data_dir+'data_CG.csv')
data.head()

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


cty_code            doc_id  supplier_id     buyer_id doc_type_code  \
0       AE  RFHLM9WX4V-89034    100369293  100244505.0           INV   
1       AE  V5HKSF85PW-36933    100369293  100244505.0           INV   
2       CN   52HNXCS9NQ-6901    100399400  100310713.0           INV   
3       CN   52HNXCS9NQ-6902    100399400  100310713.0           INV   
4       CN   52HNXCS9NQ-6968    100399400  100310713.0           INV   

      deal_id fin_flag      rec_id    doc_date doc_ccy_code  doc_ccy_amt  \
0  TF123BF562        Y  6728531788  2017-03-24          USD    113268.50   
1  TF123BF144        Y   783293897  2017-03-10          USD    106717.15   
2  TF675BK822        Y  6765119598  2017-03-14          CNY    101409.08   
3  TF675BK822        Y  6765119600  2017-03-20          CNY    197106.76   
4  TF675BK822        Y  6745980446  2017-03-15          CNY    101536.00   

   tenor     tenor_start_date    due_date  grace_period maturity_date  \
0     45  2017-03-24 00:00:00  2017-05-08             0    2017-05-08   
1     45  2017-03-10 00:00:00  2017-04-24             0    2017-04-24   
2     30  2017-03-14 00:00:00  2017-04-13           158    2017-09-18   
3     24  2017-03-20 00:00:00  2017-04-13           158    2017-09-18   
4     29  2017-03-15 00:00:00  2017-04-13           158    2017-09-18   

  fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
0  2017-03-25 00:00:00       44.0       37.0            1           USD   
1  2017-03-11 00:00:00       44.0       29.0            1           USD   
2  2017-04-10 00:00:00      161.0      116.0            1           CNY   
3  2017-04-10 00:00:00      161.0      116.0            1           CNY   
4  2017-04-10 00:00:00      161.0      116.0            1           CNY   

   pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  fin_ccy_ncp_alloc_amt  \
0       113268.50          USD                   0.0              113268.50   
1       106717.15          USD                   0.0              106717.15   
2       101409.08          CNY                   0.0              101409.08   
3       197106.76          CNY                   0.0              197106.76   
4       101536.00          CNY                   0.0              101536.00   

   fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  fin_ccy_amt  os_ccy_amt    fin_date  \
0                 0.0                0.0    113268.50         0.0  2017-03-25   
1                 0.0                0.0    106717.15         0.0  2017-03-11   
2                 0.0                0.0    101409.08         0.0  2017-04-10   
3                 0.0                0.0    197106.76         0.0  2017-04-10   
4                 0.0                0.0    101536.00         0.0  2017-04-10   

  sale_ledger_flag pool_based_flag step_code product_variant_code  rel_seq_no  \
0                Y               N      SETT               IFNB02        37.0   
1                Y               N      SETT               IFNB02        29.0   
2                Y               N      SETT               IFNB02       116.0   
3                Y               N      SETT               IFNB02       116.0   
4                Y               N      SETT               IFNB02       116.0   

    pymt_date settled_flag  late_days  finance_days  finance_due_days  isLate  \
0  2017-05-04            Y          0            40                44       0   
1  2017-04-09            Y          0            29                44       0   
2  2017-09-18            Y        158           161                 3       1   
3  2017-09-18            Y        158           161                 3       1   
4  2017-09-18            Y        158           161                 3       1   

   late_days_bucket  fin_xrate  pymt_xrate  doc_xrate    doc_usd_amt  \
0                 0   1.000000    1.000000   1.000000  113268.500000   
1                 0   1.000000    1.000000   1.000000  106717.150000   
2                 2   0.145113    0.152346   0.144957   14699.950219  

In [7]:
data_col = list(data.columns)
add_feature_col = [x for x in data_col if ("buyer_supplier" in x)|('client_' in x)]
add_feature_col

['client_id']

In [8]:
#import all:

buyer_supplier_list = [
    'buyer_supplier_3m.csv', 'buyer_supplier_6m.csv','buyer_supplier_prevyr.csv','buyer_supplier_all.csv']
buyer_supplier_dict = {}

for file in buyer_supplier_list:
    buyer_supplier_dict[file.rsplit('.csv',1)[0]] = pd.read_csv(data_dir+file)
    
for df in buyer_supplier_dict:
    data = data.merge(buyer_supplier_dict[df], on = 'doc_id', how = 'left')
    
data.head()

cty_code            doc_id  supplier_id     buyer_id doc_type_code  \
0       AE  RFHLM9WX4V-89034    100369293  100244505.0           INV   
1       AE  V5HKSF85PW-36933    100369293  100244505.0           INV   
2       CN   52HNXCS9NQ-6901    100399400  100310713.0           INV   
3       CN   52HNXCS9NQ-6902    100399400  100310713.0           INV   
4       CN   52HNXCS9NQ-6968    100399400  100310713.0           INV   

      deal_id fin_flag      rec_id    doc_date doc_ccy_code  doc_ccy_amt  \
0  TF123BF562        Y  6728531788  2017-03-24          USD    113268.50   
1  TF123BF144        Y   783293897  2017-03-10          USD    106717.15   
2  TF675BK822        Y  6765119598  2017-03-14          CNY    101409.08   
3  TF675BK822        Y  6765119600  2017-03-20          CNY    197106.76   
4  TF675BK822        Y  6745980446  2017-03-15          CNY    101536.00   

   tenor     tenor_start_date    due_date  grace_period maturity_date  \
0     45  2017-03-24 00:00:00  2017-05-08             0    2017-05-08   
1     45  2017-03-10 00:00:00  2017-04-24             0    2017-04-24   
2     30  2017-03-14 00:00:00  2017-04-13           158    2017-09-18   
3     24  2017-03-20 00:00:00  2017-04-13           158    2017-09-18   
4     29  2017-03-15 00:00:00  2017-04-13           158    2017-09-18   

  fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
0  2017-03-25 00:00:00       44.0       37.0            1           USD   
1  2017-03-11 00:00:00       44.0       29.0            1           USD   
2  2017-04-10 00:00:00      161.0      116.0            1           CNY   
3  2017-04-10 00:00:00      161.0      116.0            1           CNY   
4  2017-04-10 00:00:00      161.0      116.0            1           CNY   

   pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  fin_ccy_ncp_alloc_amt  \
0       113268.50          USD                   0.0              113268.50   
1       106717.15          USD                   0.0              106717.15   
2       101409.08          CNY                   0.0              101409.08   
3       197106.76          CNY                   0.0              197106.76   
4       101536.00          CNY                   0.0              101536.00   

   fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  fin_ccy_amt  os_ccy_amt    fin_date  \
0                 0.0                0.0    113268.50         0.0  2017-03-25   
1                 0.0                0.0    106717.15         0.0  2017-03-11   
2                 0.0                0.0    101409.08         0.0  2017-04-10   
3                 0.0                0.0    197106.76         0.0  2017-04-10   
4                 0.0                0.0    101536.00         0.0  2017-04-10   

  sale_ledger_flag pool_based_flag step_code product_variant_code  rel_seq_no  \
0                Y               N      SETT               IFNB02        37.0   
1                Y               N      SETT               IFNB02        29.0   
2                Y               N      SETT               IFNB02       116.0   
3                Y               N      SETT               IFNB02       116.0   
4                Y               N      SETT               IFNB02       116.0   

    pymt_date settled_flag  late_days  finance_days  finance_due_days  isLate  \
0  2017-05-04            Y          0            40                44       0   
1  2017-04-09            Y          0            29                44       0   
2  2017-09-18            Y        158           161                 3       1   
3  2017-09-18            Y        158           161                 3       1   
4  2017-09-18            Y        158           161                 3       1   

   late_days_bucket  fin_xrate  pymt_xrate  doc_xrate    doc_usd_amt  \
0                 0   1.000000    1.000000   1.000000  113268.500000   
1                 0   1.000000    1.000000   1.000000  106717.150000   
2                 2   0.145113    0.152346   0.144957   14699.950219  

In [9]:
client_list = ['client_3m.csv', 
'client_6m.csv', 
'client_all.csv', 
'client_prevyr.csv' ]
client_dict = {}

for file in client_list:
    client_dict[file.rsplit('.csv',1)[0]] = pd.read_csv(data_dir+file)
    
for df in client_dict:
    data = data.merge(client_dict[df], on = 'doc_id', how = 'left')
    
data.head()

cty_code            doc_id  supplier_id     buyer_id doc_type_code  \
0       AE  RFHLM9WX4V-89034    100369293  100244505.0           INV   
1       AE  V5HKSF85PW-36933    100369293  100244505.0           INV   
2       CN   52HNXCS9NQ-6901    100399400  100310713.0           INV   
3       CN   52HNXCS9NQ-6902    100399400  100310713.0           INV   
4       CN   52HNXCS9NQ-6968    100399400  100310713.0           INV   

      deal_id fin_flag      rec_id    doc_date doc_ccy_code  doc_ccy_amt  \
0  TF123BF562        Y  6728531788  2017-03-24          USD    113268.50   
1  TF123BF144        Y   783293897  2017-03-10          USD    106717.15   
2  TF675BK822        Y  6765119598  2017-03-14          CNY    101409.08   
3  TF675BK822        Y  6765119600  2017-03-20          CNY    197106.76   
4  TF675BK822        Y  6745980446  2017-03-15          CNY    101536.00   

   tenor     tenor_start_date    due_date  grace_period maturity_date  \
0     45  2017-03-24 00:00:00  2017-05-08             0    2017-05-08   
1     45  2017-03-10 00:00:00  2017-04-24             0    2017-04-24   
2     30  2017-03-14 00:00:00  2017-04-13           158    2017-09-18   
3     24  2017-03-20 00:00:00  2017-04-13           158    2017-09-18   
4     29  2017-03-15 00:00:00  2017-04-13           158    2017-09-18   

  fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
0  2017-03-25 00:00:00       44.0       37.0            1           USD   
1  2017-03-11 00:00:00       44.0       29.0            1           USD   
2  2017-04-10 00:00:00      161.0      116.0            1           CNY   
3  2017-04-10 00:00:00      161.0      116.0            1           CNY   
4  2017-04-10 00:00:00      161.0      116.0            1           CNY   

   pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  fin_ccy_ncp_alloc_amt  \
0       113268.50          USD                   0.0              113268.50   
1       106717.15          USD                   0.0              106717.15   
2       101409.08          CNY                   0.0              101409.08   
3       197106.76          CNY                   0.0              197106.76   
4       101536.00          CNY                   0.0              101536.00   

   fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  fin_ccy_amt  os_ccy_amt    fin_date  \
0                 0.0                0.0    113268.50         0.0  2017-03-25   
1                 0.0                0.0    106717.15         0.0  2017-03-11   
2                 0.0                0.0    101409.08         0.0  2017-04-10   
3                 0.0                0.0    197106.76         0.0  2017-04-10   
4                 0.0                0.0    101536.00         0.0  2017-04-10   

  sale_ledger_flag pool_based_flag step_code product_variant_code  rel_seq_no  \
0                Y               N      SETT               IFNB02        37.0   
1                Y               N      SETT               IFNB02        29.0   
2                Y               N      SETT               IFNB02       116.0   
3                Y               N      SETT               IFNB02       116.0   
4                Y               N      SETT               IFNB02       116.0   

    pymt_date settled_flag  late_days  finance_days  finance_due_days  isLate  \
0  2017-05-04            Y          0            40                44       0   
1  2017-04-09            Y          0            29                44       0   
2  2017-09-18            Y        158           161                 3       1   
3  2017-09-18            Y        158           161                 3       1   
4  2017-09-18            Y        158           161                 3       1   

   late_days_bucket  fin_xrate  pymt_xrate  doc_xrate    doc_usd_amt  \
0                 0   1.000000    1.000000   1.000000  113268.500000   
1                 0   1.000000    1.000000   1.000000  106717.150000   
2                 2   0.145113    0.152346   0.144957   14699.950219  

In [10]:
repeated_cols = [x for x in list(data.columns) if (x[-2:]=='_x')|(x[-2:]=='_y')]
repeated_cols

['Unnamed: 0_x',
 'rownum_x',
 'Unnamed: 0_y',
 'rownum_y',
 'buyer_supplier_season_x',
 'Unnamed: 0_x',
 'rownum_x',
 'Unnamed: 0_y',
 'rownum_y',
 'buyer_supplier_season_y',
 'rownum_x',
 'rownum_y',
 'client_season_x',
 'rownum_x',
 'rownum_y',
 'client_season_y']

In [11]:
repeated_cols = [x for x in repeated_cols if x not in ['buyer_supplier_season_x','client_season_x']]
data.drop(repeated_cols, axis = 1, inplace = True)
data.columns

Index([u'cty_code', u'doc_id', u'supplier_id', u'buyer_id', u'doc_type_code',
       u'deal_id', u'fin_flag', u'rec_id', u'doc_date', u'doc_ccy_code',
       ...
       u'client_6m_term', u'client_6m_late_days', u'client_6m_isLate',
       u'client_prevyr_inv_cnt', u'client_prevyr_ave_fin_amt',
       u'client_prevyr_std_fin_amt', u'client_prevyr_late_days',
       u'client_prevyr_isLate', u'client_prevyr_dr', u'client_prevyr_term'],
      dtype='object', length=123)

In [12]:
data.head()

cty_code            doc_id  supplier_id     buyer_id doc_type_code  \
0       AE  RFHLM9WX4V-89034    100369293  100244505.0           INV   
1       AE  V5HKSF85PW-36933    100369293  100244505.0           INV   
2       CN   52HNXCS9NQ-6901    100399400  100310713.0           INV   
3       CN   52HNXCS9NQ-6902    100399400  100310713.0           INV   
4       CN   52HNXCS9NQ-6968    100399400  100310713.0           INV   

      deal_id fin_flag      rec_id    doc_date doc_ccy_code  doc_ccy_amt  \
0  TF123BF562        Y  6728531788  2017-03-24          USD    113268.50   
1  TF123BF144        Y   783293897  2017-03-10          USD    106717.15   
2  TF675BK822        Y  6765119598  2017-03-14          CNY    101409.08   
3  TF675BK822        Y  6765119600  2017-03-20          CNY    197106.76   
4  TF675BK822        Y  6745980446  2017-03-15          CNY    101536.00   

   tenor     tenor_start_date    due_date  grace_period maturity_date  \
0     45  2017-03-24 00:00:00  2017-05-08             0    2017-05-08   
1     45  2017-03-10 00:00:00  2017-04-24             0    2017-04-24   
2     30  2017-03-14 00:00:00  2017-04-13           158    2017-09-18   
3     24  2017-03-20 00:00:00  2017-04-13           158    2017-09-18   
4     29  2017-03-15 00:00:00  2017-04-13           158    2017-09-18   

  fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
0  2017-03-25 00:00:00       44.0       37.0            1           USD   
1  2017-03-11 00:00:00       44.0       29.0            1           USD   
2  2017-04-10 00:00:00      161.0      116.0            1           CNY   
3  2017-04-10 00:00:00      161.0      116.0            1           CNY   
4  2017-04-10 00:00:00      161.0      116.0            1           CNY   

   pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  fin_ccy_ncp_alloc_amt  \
0       113268.50          USD                   0.0              113268.50   
1       106717.15          USD                   0.0              106717.15   
2       101409.08          CNY                   0.0              101409.08   
3       197106.76          CNY                   0.0              197106.76   
4       101536.00          CNY                   0.0              101536.00   

   fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  fin_ccy_amt  os_ccy_amt    fin_date  \
0                 0.0                0.0    113268.50         0.0  2017-03-25   
1                 0.0                0.0    106717.15         0.0  2017-03-11   
2                 0.0                0.0    101409.08         0.0  2017-04-10   
3                 0.0                0.0    197106.76         0.0  2017-04-10   
4                 0.0                0.0    101536.00         0.0  2017-04-10   

  sale_ledger_flag pool_based_flag step_code product_variant_code  rel_seq_no  \
0                Y               N      SETT               IFNB02        37.0   
1                Y               N      SETT               IFNB02        29.0   
2                Y               N      SETT               IFNB02       116.0   
3                Y               N      SETT               IFNB02       116.0   
4                Y               N      SETT               IFNB02       116.0   

    pymt_date settled_flag  late_days  finance_days  finance_due_days  isLate  \
0  2017-05-04            Y          0            40                44       0   
1  2017-04-09            Y          0            29                44       0   
2  2017-09-18            Y        158           161                 3       1   
3  2017-09-18            Y        158           161                 3       1   
4  2017-09-18            Y        158           161                 3       1   

   late_days_bucket  fin_xrate  pymt_xrate  doc_xrate    doc_usd_amt  \
0                 0   1.000000    1.000000   1.000000  113268.500000   
1                 0   1.000000    1.000000   1.000000  106717.150000   
2                 2   0.145113    0.152346   0.144957   14699.950219  

In [13]:
[x for x in data.columns if 'product' in x]

['product_variant_code', 'product_']

In [14]:
# data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data['first_client_invoice']=np.where(data.client_past_inv_cnt==0,1,0)

In [15]:
len(pd.unique(data.supplier_id)), len(pd.unique(data.buyer_id))

(377, 850)

In [16]:
len(pd.unique(data.client_id))

273

In [17]:
data.describe()

supplier_id      buyer_id   doc_ccy_amt          tenor   grace_period  \
count  2.966210e+05  2.966210e+05  2.966210e+05  296621.000000  296621.000000   
mean   1.002386e+08  1.002482e+08  1.075819e+06      57.219438       0.540757   
std    1.135222e+05  1.013388e+05  2.703321e+07      29.304096       5.963672   
min    1.000000e+08  1.000005e+08  1.400000e-01       0.000000       0.000000   
25%    1.002159e+08  1.001983e+08  8.175200e+02      30.000000       0.000000   
50%    1.002928e+08  1.003068e+08  1.369582e+04      45.000000       0.000000   
75%    1.003069e+08  1.003068e+08  5.777740e+04      90.000000       0.000000   
max    1.004228e+08  1.004310e+08  8.323897e+09     360.000000     158.000000   

           fin_tenor      to_seq_no    from_seq_no  pymt_alloc_amt  \
count  296620.000000  296589.000000  296621.000000    2.966210e+05   
mean       69.722433    6113.652620    5855.242198    1.076426e+06   
std        27.002655   15552.149771   15214.376367    2.702758e+07   
min         0.000000       2.000000       1.000000    1.000000e-02   
25%        51.000000      56.000000       1.000000    8.023700e+02   
50%        62.000000    3528.000000    3407.000000    1.358301e+04   
75%        92.000000    6314.000000    5830.000000    5.739091e+04   
max       360.000000   99314.000000   99312.000000    8.323897e+09   

       fin_ccy_cp_alloc_amt  fin_ccy_ncp_alloc_amt  fin_ccy_ncp_os_amt  \
count          2.966210e+05           2.966210e+05       296621.000000   
mean           2.512089e+05           8.243763e+05            1.854485   
std            4.776715e+06           2.660494e+07          178.535650   
min            0.000000e+00           0.000000e+00            0.000000   
25%            1.708700e+02           0.000000e+00            0.000000   
50%            2.608530e+03           0.000000e+00            0.000000   
75%            2.762701e+04           0.000000e+00            0.000000   
max            4.994590e+08           8.323897e+09        17189.910000   

       fin_ccy_cp_os_amt   fin_ccy_amt     os_ccy_amt     rel_seq_no  \
count      296621.000000  2.966210e+05  296621.000000  296589.000000   
mean           16.690368  1.075585e+06      18.544854    6113.652620   
std          1606.820851  2.702270e+07    1785.356501   15552.149771   
min             0.000000  1.300000e-01       0.000000       2.000000   
25%             0.000000  8.222400e+02       0.000000      56.000000   
50%             0.000000  1.387540e+04       0.000000    3528.000000   
75%             0.000000  5.906463e+04       0.000000    6314.000000   
max        154709.190000  8.323897e+09  171899.100000   99314.000000   

           late_days   finance_days  finance_due_days         isLate  \
count  296621.000000  296621.000000     296621.000000  296621.000000   
mean        3.901332      47.780397         44.548731       0.584696   
std         9.313458      25.102624         25.645612       0.492775   
min         0.000000       0.000000          0.000000       0.000000   
25%         0.000000      25.000000         22.000000       0.000000   
50%         1.000000      44.000000         41.000000       1.000000   
75%         5.000000      69.000000         66.000000       1.000000   
max       158.000000     270.000000        270.000000       1.000000   

       late_days_bucket      fin_xrate     pymt_xrate      doc_xrate  \
count     296621.000000  296621.000000  296621.000000  296621.000000   
mean           0.602007       0.463953       0.474427       0.473425   
std            0.523659       0.488360       0.490078       0.489019   
min            0.000000       0.000075       0.000075       0.000075   
25%            0.000000       0.029017       0.029033       0.029017   
50%            1.000000       0.030382       0.031337       0.030382   
75%            1.000000       1.000000       1.000000       1.000000   
max            2.000000       1.558917       1.557793       1.562907   

        doc_usd_amt   fin_usd_amt

In [18]:
Counter(data.CG)

Counter({'10A': 7920,
         '10B': 2929,
         '11A': 641,
         '11B': 2671,
         '11C': 4934,
         '14': 1,
         '2B': 2,
         '3B': 397,
         '4A': 695,
         '4B': 670,
         '5A': 1449,
         '5B': 14963,
         '6A': 11706,
         '6B': 97231,
         '7A': 42603,
         '7B': 30292,
         '8A': 19000,
         '8B': 10960,
         '9A': 20776,
         '9B': 18802,
         'no_CG': 7979})

In [19]:
np.sum(data.first_client_invoice)

171L

In [20]:
data[data.client_past_inv_cnt==0][['client_id', 'doc_id', 'client_past_inv_cnt', 'doc_date']]

client_id            doc_id  client_past_inv_cnt    doc_date
2379    100397768     6YHS7WN5NF-66                  0.0  2017-04-13
3628    100397767  NZJ4D6KZVP-75098                  0.0  2017-06-07
3909    100405632  VQJ2QVDDJD-61060                  0.0  2017-05-10
14225   100421630  4JJHFW54Q2-36534                  0.0  2017-10-18
14226   100421630  4JJHFW54Q2-36535                  0.0  2017-10-18
14227   100421630  4JJHFW54Q2-36536                  0.0  2017-10-18
14228   100421630  4JJHFW54Q2-36537                  0.0  2017-10-18
14964   100424264  5PJQWF29XD-52967                  0.0  2017-12-22
14965   100424264  5PJQWF29XD-52968                  0.0  2017-12-22
15034   100416986  5YJ7PWVDDN-64509                  0.0  2017-08-19
15035   100416986  5YJ7PWVDDN-64510                  0.0  2017-08-19
15036   100416986  5YJ7PWVDDN-64511                  0.0  2017-08-19
15037   100416986  5YJ7PWVDDN-64512                  0.0  2017-08-19
15038   100416986  5YJ7PWVDDN-64513                  0.0  2017-08-19
15069   100421183  5ZJDY1C13D-34962                  0.0  2017-09-29
17512   100407888  D4HTVVNCNS-38574                  0.0  2017-05-09
17513   100407888  D4HTVVNCNS-38576                  0.0  2017-05-09
17562   100424258  D5JMKMWQK8-78159                  0.0  2017-11-27
17564   100424258  D5JMKMWQK8-79959                  0.0  2017-11-27
17728   100413105   DQJ1BX1T9N-6418                  0.0  2017-06-24
17729   100413105   DQJ1BX1T9N-6420                  0.0  2017-06-24
19143   100421256   G9JFWS8QSF-5451                  0.0  2017-10-09
19466   100424263  GTJMP266GZ-90492                  0.0  2017-11-27
19478   100424264  GZJQT39N8C-63205                  0.0  2017-12-22
19479   100424264  GZJQT39N8C-63206                  0.0  2017-12-22
19601   100421183  HCJFJ45MN1-34067                  0.0  2017-09-29
24372   100409113  NRJ4NR685M-20778                  0.0  2017-07-09
29199   100403066  Q5HN1BW9DZ-96428                  0.0  2017-03-23
29200   100403066  Q5HN1BW9DZ-96430                  0.0  2017-03-23
29201   100403066  Q5HN1BW9DZ-96429                  0.0  2017-03-23
...           ...               ...                  ...         ...
150596  100415769  FQJPG6M429-63082                  0.0  2017-10-05
150597  100415769  FQJPG6M429-63083                  0.0  2017-10-05
150598  100415769  FQJPG6M429-63084                  0.0  2017-10-05
150599  100415769  FQJPG6M429-63085                  0.0  2017-10-05
150600  100415769  FQJPG6M429-63086                  0.0  2017-10-05
150601  100415769  FQJPG6M429-63087                  0.0  2017-10-05
150602  100415769  FQJPG6M429-63088                  0.0  2017-10-05
150603  100415769  FQJPG6M429-63089                  0.0  2017-10-05
150604  100415769  FQJPG6M429-63090                  0.0  2017-10-05
150605  100415769  FQJPG6M429-63091                  0.0  2017-10-05
150606  100415769  FQJPG6M429-63092                  0.0  2017-10-05
150607  100415769  FQJPG6M429-63093                  0.0  2017-10-05
150608  100415769  FQJPG6M429-63094                  0.0  2017-10-05
150609  100415769  FQJPG6M429-63095                  0.0  2017-10-05
150610  100415769  FQJPG6M429-63096                  0.0  2017-10-05
150611  100415769  FQJPG6M429-63097                  0.0  2017-10-05
150612  100415769  FQJPG6M429-63098                  0.0  2017-10-05
150613  100415769  FQJPG6M429-63099                  0.0  2017-10-05
150614  100415769  FQJPG6M429-63100                  0.0  2017-10-05
150615  100415769  FQJPG6M429-63101                  0.0  2017-10-05
154297  100114483  GPJQJYGB7C-67085                  0.0  2017-11-21
154298  100114483  GPJQJYGB7C-67086                  0.0  2017-11-21
170948  100397816  MNJDBLV6BJ-39296                  0.0  2017-08-07
170949  100397816  MNJDBLV6BJ-39297                  0.0  2017-08-07
177932  100063132  QCJD89K7VL-38737                  0.0  2017-07-19
177933  100063132  QCJD89K7VL-38738                  0.0  2017-07-19
177958

#### multiple transactions tagged as "first" - same client, same date

In [21]:
data.columns

Index([u'cty_code', u'doc_id', u'supplier_id', u'buyer_id', u'doc_type_code',
       u'deal_id', u'fin_flag', u'rec_id', u'doc_date', u'doc_ccy_code',
       ...
       u'client_6m_late_days', u'client_6m_isLate', u'client_prevyr_inv_cnt',
       u'client_prevyr_ave_fin_amt', u'client_prevyr_std_fin_amt',
       u'client_prevyr_late_days', u'client_prevyr_isLate',
       u'client_prevyr_dr', u'client_prevyr_term', u'first_client_invoice'],
      dtype='object', length=124)

### RICI index

In [22]:
RICI = pd.read_csv('/home/flowcast/data/python/external_data/RICI_data.csv')
RICI.head()

Date    Value
0  7/31/98  1000.00
1  8/10/98   970.75
2  8/11/98   956.80
3  8/12/98   957.82
4  8/13/98   969.54

In [23]:
RICI.tail()

Date    Value
4469  2/26/18  2485.53
4470  2/27/18  2470.66
4471  2/28/18  2452.59
4472   3/1/18  2446.86
4473   3/2/18  2446.86

In [24]:
RICI['Date']=[pd.to_datetime(x) for x in RICI.Date]
type(RICI.Date[4])

pandas._libs.tslib.Timestamp

In [25]:
data['doc_date'] = [pd.to_datetime(x) for x in data.doc_date]
type(data.doc_date[5])

pandas._libs.tslib.Timestamp

In [26]:
RICI.rename(columns={'Value':'RICI_index', 'Date':'doc_date'}, inplace=True)
data = data.merge(RICI, how = 'left', on = 'doc_date')
data.tail()

cty_code           doc_id  supplier_id     buyer_id doc_type_code  \
296616       US  ZCHR3NLSF6-8119    100363725  100199860.0           INV   
296617       US  ZCHR3NLSF6-8120    100363725  100199860.0           INV   
296618       US  ZCHR3NLSF6-8121    100363725  100199860.0           INV   
296619       US  ZCHR3NLSF6-8132    100363725  100364871.0           INV   
296620       US  ZCHR3NLSF6-8167    100363725  100364922.0           INV   

           deal_id fin_flag      rec_id   doc_date doc_ccy_code  doc_ccy_amt  \
296616  TF777R0093        Y  6851359338 2017-04-11          USD     20075.49   
296617  TF777R0093        Y  6851359336 2017-04-11          USD     20865.82   
296618  TF777R0093        Y  6851359342 2017-04-13          USD     20035.01   
296619  TF777R0093        Y  6851359348 2017-04-11          USD     16309.40   
296620  TF777R0093        Y  6851359360 2017-04-11          USD     34300.00   

        tenor     tenor_start_date    due_date  grace_period maturity_date  \
296616     35  2017-04-11 00:00:00  2017-05-16             0    2017-06-15   
296617     35  2017-04-11 00:00:00  2017-05-16             0    2017-06-15   
296618     35  2017-04-13 00:00:00  2017-05-18             0    2017-06-19   
296619     30  2017-04-11 00:00:00  2017-05-11             0    2017-06-12   
296620     30  2017-04-11 00:00:00  2017-05-11             0    2017-06-12   

       fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
296616  2017-04-11 00:00:00       65.0     5310.0         4468           USD   
296617  2017-04-11 00:00:00       65.0     5310.0         4468           USD   
296618  2017-04-13 00:00:00       67.0     5375.0         4468           USD   
296619  2017-04-11 00:00:00       62.0     5229.0         4451           USD   
296620  2017-04-11 00:00:00       62.0     5225.0         4447           USD   

        pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  \
296616        18067.94          USD              18067.94   
296617        18779.24          USD              18779.24   
296618        18031.51          USD              18031.51   
296619        14678.46          USD              14678.46   
296620        30870.00          USD              30870.00   

        fin_ccy_ncp_alloc_amt  fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  \
296616                    0.0                 0.0                0.0   
296617                    0.0                 0.0                0.0   
296618                    0.0                 0.0                0.0   
296619                    0.0                 0.0                0.0   
296620                    0.0                 0.0                0.0   

        fin_ccy_amt  os_ccy_amt    fin_date sale_ledger_flag pool_based_flag  \
296616     18067.94         0.0  2017-04-19                Y               N   
296617     18779.24         0.0  2017-04-19                Y               N   
296618     18031.51         0.0  2017-04-19                Y               N   
296619     14678.46         0.0  2017-04-19                Y               N   
296620     30870.00         0.0  2017-04-19                Y               N   

       step_code product_variant_code  rel_seq_no   pymt_date settled_flag  \
296616      SETT               RSRV01      5310.0  2017-06-15            Y   
296617      SETT               RSRV01      5310.0  2017-06-15            Y   
296618      SETT               RSRV01      5375.0  2017-06-19            Y   
296619      SETT               RSRV01      5229.0  2017-06-12            Y   
296620      SETT               RSRV01      5225.0  2017-06-12            Y   

        late_days  finance_days  finance_due_days  isLate  late_days_bucket  \
296616         30            57                27       1                 1   
296617         30            57                27       1                 1   
296618         32            61                29       1                 2   
296619         32            54                22    

### Day of Week

In [27]:
data.doc_date[4].weekday()
# data.due_date

2

In [28]:
data['due_date_dow'] =[pd.to_datetime(data.due_date[i]).weekday() for i in range(len(data))]
data['maturity_date_dow'] = [pd.to_datetime(data.maturity_date[i]).weekday() for i in range(len(data))]
data['buyer_centric_flag'] = [1 if x in ['IFNB', 'VSRP'] else 0 for x in data['product_']]
data.tail()

cty_code           doc_id  supplier_id     buyer_id doc_type_code  \
296616       US  ZCHR3NLSF6-8119    100363725  100199860.0           INV   
296617       US  ZCHR3NLSF6-8120    100363725  100199860.0           INV   
296618       US  ZCHR3NLSF6-8121    100363725  100199860.0           INV   
296619       US  ZCHR3NLSF6-8132    100363725  100364871.0           INV   
296620       US  ZCHR3NLSF6-8167    100363725  100364922.0           INV   

           deal_id fin_flag      rec_id   doc_date doc_ccy_code  doc_ccy_amt  \
296616  TF777R0093        Y  6851359338 2017-04-11          USD     20075.49   
296617  TF777R0093        Y  6851359336 2017-04-11          USD     20865.82   
296618  TF777R0093        Y  6851359342 2017-04-13          USD     20035.01   
296619  TF777R0093        Y  6851359348 2017-04-11          USD     16309.40   
296620  TF777R0093        Y  6851359360 2017-04-11          USD     34300.00   

        tenor     tenor_start_date    due_date  grace_period maturity_date  \
296616     35  2017-04-11 00:00:00  2017-05-16             0    2017-06-15   
296617     35  2017-04-11 00:00:00  2017-05-16             0    2017-06-15   
296618     35  2017-04-13 00:00:00  2017-05-18             0    2017-06-19   
296619     30  2017-04-11 00:00:00  2017-05-11             0    2017-06-12   
296620     30  2017-04-11 00:00:00  2017-05-11             0    2017-06-12   

       fin_tenor_start_date  fin_tenor  to_seq_no  from_seq_no pymt_ccy_code  \
296616  2017-04-11 00:00:00       65.0     5310.0         4468           USD   
296617  2017-04-11 00:00:00       65.0     5310.0         4468           USD   
296618  2017-04-13 00:00:00       67.0     5375.0         4468           USD   
296619  2017-04-11 00:00:00       62.0     5229.0         4451           USD   
296620  2017-04-11 00:00:00       62.0     5225.0         4447           USD   

        pymt_alloc_amt fin_ccy_code  fin_ccy_cp_alloc_amt  \
296616        18067.94          USD              18067.94   
296617        18779.24          USD              18779.24   
296618        18031.51          USD              18031.51   
296619        14678.46          USD              14678.46   
296620        30870.00          USD              30870.00   

        fin_ccy_ncp_alloc_amt  fin_ccy_ncp_os_amt  fin_ccy_cp_os_amt  \
296616                    0.0                 0.0                0.0   
296617                    0.0                 0.0                0.0   
296618                    0.0                 0.0                0.0   
296619                    0.0                 0.0                0.0   
296620                    0.0                 0.0                0.0   

        fin_ccy_amt  os_ccy_amt    fin_date sale_ledger_flag pool_based_flag  \
296616     18067.94         0.0  2017-04-19                Y               N   
296617     18779.24         0.0  2017-04-19                Y               N   
296618     18031.51         0.0  2017-04-19                Y               N   
296619     14678.46         0.0  2017-04-19                Y               N   
296620     30870.00         0.0  2017-04-19                Y               N   

       step_code product_variant_code  rel_seq_no   pymt_date settled_flag  \
296616      SETT               RSRV01      5310.0  2017-06-15            Y   
296617      SETT               RSRV01      5310.0  2017-06-15            Y   
296618      SETT               RSRV01      5375.0  2017-06-19            Y   
296619      SETT               RSRV01      5229.0  2017-06-12            Y   
296620      SETT               RSRV01      5225.0  2017-06-12            Y   

        late_days  finance_days  finance_due_days  isLate  late_days_bucket  \
296616         30            57                27       1                 1   
296617         30            57                27       1                 1   
296618         32            61                29       1                 2   
296619         32            54                22    

In [ ]:
#ccy features

ccy_list = [
    'ccy_features_vector.csv', 'ccy_features_vol.csv'  ]

for ccy_file in ccy_list:
    df = pd.read_csv(data_dir+ccy_file)
    df

In [43]:
#ccy features

ccy_list = [
    'ccy_features_vector.csv', 'ccy_features_vol.csv'  ]
# suffix = ['vector','vol']
ccy_dict = {}

for file in ccy_list:
    ccy_dict[file.rsplit('.csv',1)[0]] = pd.read_csv(data_dir+file)
    
# ccy_dict.keys()
# for df in ccy_dict:
#     df.rename(columns = {x:x+})
#     data = data.merge(ccy_dict[df], on = 'doc_id', how = 'left')
    
# data.tail()

for k,v in ccy_dict.iteritems():
#     print('old',v.columns)
    suffix = k.replace('ccy_features',"")
    v= v[v.columns[1:]]
    v.rename(columns = {x:x+suffix for x in v.columns if x[:3]=='ccy'}, inplace =True)
    data = data.merge(v, on = 'doc_id', how = 'left')
#     print('new',v.columns)
#     print(v.tail())

In [44]:
data.rename(columns={'sic_code':'industry_code'}, inplace=True)

In [45]:
for col in data.columns: print(col)

cty_code
doc_id
supplier_id
buyer_id
doc_type_code
deal_id
fin_flag
rec_id
doc_date
doc_ccy_code
doc_ccy_amt
tenor
tenor_start_date
due_date
grace_period
maturity_date
fin_tenor_start_date
fin_tenor
to_seq_no
from_seq_no
pymt_ccy_code
pymt_alloc_amt
fin_ccy_code
fin_ccy_cp_alloc_amt
fin_ccy_ncp_alloc_amt
fin_ccy_ncp_os_amt
fin_ccy_cp_os_amt
fin_ccy_amt
os_ccy_amt
fin_date
sale_ledger_flag
pool_based_flag
step_code
product_variant_code
rel_seq_no
pymt_date
settled_flag
late_days
finance_days
finance_due_days
isLate
late_days_bucket
fin_xrate
pymt_xrate
doc_xrate
doc_usd_amt
fin_usd_amt
os_usd_amt
pymt_usd_amt
dr
dr_bucket
product_
client_id
le_id
lid_isic_cd_val
industry_code
CG
buyer_supplier_yrs
buyer_supplier_past_inv_cnt
buyer_supplier_annual_inv_cnt
buyer_supplier_ave_fin_amt
buyer_supplier_ave_annual_fin_amt
buyer_supplier_std_fin_amt
buyer_supplier_late_days
buyer_supplier_isLate
buyer_supplier_dr
buyer_supplier_term
buyer_supplier_cty
buyer_supplier_prevyr_inv_cnt
buyer_supplier

In [46]:
data.to_csv(data_dir+'data_for_model.csv', index=False)